## Importing

In [40]:
import load_data as ld
import dataset_utilities as du

Let's get some preprocessed data from the 2010 movie dataset.  The multithreaded module only contains one function to call, and it is a convenience method to parallel preprocess the 2010 data.  

In [41]:
ds_list = ld.get_2010_preprocessed_data(num_subjects=5, mask_path='masks/aal_l_hippocampus_3x3x3.nii', num_threads=5, combine=False)

 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *


Now we can take a look at what we got. 

In [42]:
print("Gathered Datasets for {0} subjects".format(len(ds_list)))
print("Shape of each Dataset: {0}".format(ds_list[0].shape))

Gathered Datasets for 5 subjects
Shape of each Dataset: (633, 318)


So we have a list of 5 subjects Dataset objects.  Each Dataset contains the preprocessed and spliced runs with 633 time points and 318 voxels inside of the left hippocampus mask we supplied.

Let's take just one of the Datasets and examine the time series for the voxel 50.

In [43]:
ds = ds_list[0]
du.voxel_plot(ds, 50)

This voxel_plot() function just quickly lets us examine the time series for a single voxel.  You can see that the runs of have been detrended and z-scored.  Now lets do something a little more interesting.
***

## Searchlights

In order to run a searchlight analysis between multiple subjects, we need to combine the datasets into one single
Dataset object to feed into pymvpa's searchlight function.  To do this we have two options:

1: You already knew from the start that you wanted the datasets combined, so in multithreaded we set the combine keyword to True (which it is by default)

In [44]:
combined_ds = ld.get_2010_preprocessed_data(num_subjects=5, mask_path='masks/aal_l_hippocampus_3x3x3.nii', num_threads=5, combine=True)

 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *


2: We take the dataset list we got earlier and use the following function to combine them

In [45]:
cds = du.combine_datasets(ds_list)

Either way we get the same single Dataset object that is ready to be passed to a searchlight method.

In [46]:
print(combined_ds.shape)
print(cds.shape)

(5, 318, 633)
(5, 318, 633)


Let's run a simple Pearson's correlation first, with single voxel univariate comparisons.  We do this by setting the
searchlight radius to 0.

In [47]:
results = du.run_searchlight(cds, metric='correlation', radius=0, n_cpu=4)
print(results.__class__)
print(results.shape)

<class 'mvpa2.datasets.base.Dataset'>
(1, 318)


So here we got a Dataset object with 318 average Pearson's correlation values for all pairwise combinations of subjects.  From here we can easily export the data to a nifti file and look at the results in AFNI or another fmri viewing tool.  A simple function to do this is the following:

In [48]:
#du.export_to_nifti(results, 'left_hippo_corr_r0')